In [2]:
import os
from dotenv import load_dotenv
from web3 import Web3

from story_protocol_python_sdk import StoryClient

In [3]:
load_dotenv()
private_key = os.getenv('WALLET_PRIVATE_KEY')
rpc_url = os.getenv('RPC_PROVIDER_URL')

# Initialize Web3
web3 = Web3(Web3.HTTPProvider(rpc_url))

# Set up the account with the private key
account = web3.eth.account.from_key(private_key)
story_client = StoryClient(web3, account, 11155111)

In [33]:
response = story_client.IPAsset.register(
    token_contract="0xd516482bef63Ff19Ed40E4C6C2e626ccE04e19ED", # your NFT contract address
    token_id=12 # your NFT token ID
)

print(f"Root IPA created at transaction hash {response['txHash']}")
print(f"IPA ID: {response['ipId']}")

Root IPA created at transaction hash None
IPA ID: 0x234B4a0bBC330598aCe669616B9Eff465bd4F3ea


In [32]:
response2 = story_client.License.registerCommercialUsePIL(
    minting_fee=1,
    currency="0xB132A6B7AE652c974EE1557A3521D53d18F6739f",
    royalty_policy="0xAAbaf349C7a2A84564F9CC4Ac130B3f19A718E86"
)

print(f"PIL Terms registered at transaction hash {response2['txHash']} License Terms ID: {response2['licenseTermsId']}")

KeyError: 'txHash'

In [30]:
response

{'licenseTermsId': 3}

In [ ]:
response3 = story_client.License.registerCommercialRemixPIL(
    minting_fee=1,
    currency="0xB132A6B7AE652c974EE1557A3521D53d18F6739f",
    commercial_rev_share=10,
    royalty_policy="0xAAbaf349C7a2A84564F9CC4Ac130B3f19A718E86"
)

print(f"PIL Terms registered at transaction hash {response3['txHash']} License Terms ID: {response3['licenseTermsId']}")

KeyError: 'txHash'

In [ ]:
response

{'licenseTermsId': 60}

In [ ]:
response = story_client.License.registerNonComSocialRemixingPIL()

print(f"PIL Terms registered at transaction hash {response['txHash']} License Terms ID: {response['licenseTermsId']}")

KeyError: 'txHash'

In [26]:
response

{'licenseTermsId': 3}

In [36]:
try:
  response_a = story_client.License.attachLicenseTerms(
    ip_id="0x234B4a0bBC330598aCe669616B9Eff465bd4F3ea", 
    license_template="0x260B6CB6284c89dbE660c0004233f7bB99B5edE7", 
    license_terms_id=3
  )
  print(f"Attached License Terms to IPA at transaction hash {response_a['txHash']}.")
except Exception as e:
  print("License Terms already attached to this IPA")

License Terms already attached to this IPA


In [5]:
import cv2
import numpy as np

In [3]:
def concatenate_images_opencv(image_paths, horizontal=True):
    images = [cv2.imread(img_path) for img_path in image_paths]
    
    if horizontal:
        concatenated_image = np.hstack(images)
    else:
        concatenated_image = np.vstack(images)
    
    return concatenated_image

# Example usage
paths = ['hello_bye.png', 'hello_bye_copy.png', 'hello_bye_copy.png']
result_image = concatenate_images_opencv(paths, horizontal=False)
cv2.imwrite('concatenated_image_vertical.jpg', result_image)

True

In [6]:
img = cv2.imread('hello_bye.png')
img.shape

(3329, 6300, 3)

In [41]:
def grab_background_color(image, approach):
    h, w, _ = image.shape
    majority_vote = dict()
    if approach == "corners_and_edges":
        for i in range(h):
            try:
                majority_vote[tuple(image[i, 0, :])] += 1
            except KeyError:
                majority_vote[tuple(image[i, 0, :])] = 1

            try:
                majority_vote[tuple(image[i, -1, :])] += 1
            except KeyError:
                majority_vote[tuple(image[i, -1, :])] = 1
                 
        for j in range(1, w-1):
            try:
                majority_vote[tuple(image[0, j, :])] += 1
            except KeyError:
                majority_vote[tuple(image[0, j, :])] = 1

            try:
                majority_vote[tuple(image[-1, j, :])] += 1
            except KeyError:
                majority_vote[tuple(image[-1, j, :])] = 1
    
    max_count, max_combi = 0, None
    for color_combination, count in majority_vote.items():
        if count > max_count: 
            max_count= count
            max_combi = color_combination
    return max_combi

def concatenate_images_inline(images, horizontal=True):
    if horizontal:
        concatenated_image = np.hstack(images)
    else:
        concatenated_image = np.vstack(images)
    
    return concatenated_image

In [30]:
import json
CURRENT_PATH = os.getenv('CURRENT_PATH')
with open(CURRENT_PATH + "/factory/context/on_chain_security/materials.json", "r") as f:
        materials = json.load(f)
        print(materials)

{'bottom_text': [{'emotion': 'angry', 'sitution': 'rugged', 'text': 'Write down what address, what chain, and report to XYZ what happened'}, {'emotion': 'angry', 'sitution': 'drained', 'text': 'Write down what Chain, the address that drained and keep on eye on what projects they touch'}, {'emotion': 'shame', 'sitution': 'rugged', 'text': 'Write down what address, what chain, and report to XYZ what happened'}, {'emotion': 'shame', 'sitution': 'drained', 'text': 'Write down what Chain, the address that drained and keep on eye on what projects they touch'}, {'emotion': 'Happiness', 'sitution': 'fund_retrieval_successful', 'text': 'Remember to stay alert and tell others who helped you and how!'}], 'top_text': [{'emotion': 'angry', 'sitution': 'rugged', 'text': 'Rugged? Take a deep breath first.'}, {'emotion': 'angry', 'sitution': 'drained', 'text': 'Drained? Take a deep breath first.'}, {'emotion': 'shame', 'sitution': 'rugged', 'text': 'It happens to everyone...'}, {'emotion': 'shame', 's

In [40]:
def add_text_alignment_check(
        image_path: list, image: np.array, inline: bool,
        text: str, 
        position: tuple, alignment: str,
        font, font_scale: int, font_color: tuple, font_thickness: int
        ):
    """
    add text as well as where it's aligned. By default it's left aligned from the position
    """
    if alignment == "right":
        (text_w, text_h), _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        position = (position[0]-text_w, position[1]+text_h)
    elif alignment == "center":
        (text_w, text_h), _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        position = (position[0]-text_w//2, position[1]+text_h//2)

    if inline is True:
        return __add_text_to_image_inline(image, text, position, font, font_scale, font_color, font_thickness)
    else:
        return __add_text_to_image_path(image_path, text, position, font, font_scale, font_color, font_thickness)

def __add_text_to_image_inline(image, text, position, font=cv2.FONT_ITALIC, font_scale=1, font_color=(255, 0, 0), font_thickness=50):
    # Add text to the image
    cv2.putText(image, text, position, font, font_scale, font_color, font_thickness)

    return image

def __add_text_to_image_path(image_path, text, position, font=cv2.FONT_ITALIC, font_scale=1, font_color=(255, 0, 0), font_thickness=50):
    # Read the image
    image = cv2.imread(image_path)

    # Add text to the image
    cv2.putText(image, text, position, font, font_scale, font_color, font_thickness)

    return image

In [42]:
def create_pad(color, height, width, depth=3):
    return np.ones((height, width, depth))*color

def add_padding_for_text(img, approach, original_to_pad_height_ratio):
    pad_color = grab_background_color(img, approach)
    h, w, d = img.shape
    padding = create_pad(pad_color, h//original_to_pad_height_ratio, w, d)
    return concatenate_images_inline(images=[padding, img, padding], horizontal=False)

In [36]:
imgs, backgrounds = [], []
for index, pic, bottom, top in enumerate(materials['pic_path'], materials['bottom_text'], materials['top_text']):
    path, tt, bt = pic['text'], top['text'], bottom['text']
    img = cv2.imread(path)

    padded = add_padding_for_text(img, "corners_and_edges", 3)
    add_text_alignment_check(
        image_path=None, image=padded, inline=True,
        text=tt, 
        position="", alignment="center",
        font, font_scale: int, font_color: tuple, font_thickness: int
        )

    cv2.imwrite("test_{}.png".format(index), padded)
    cv2.imwrite("original_{}.png".format(index), img)
    # imgs.append(img)
    # backgrounds.append(bck)


In [13]:
grab_background_color(img, "corners_and_edges")

(255, 255, 255)

def add_text_alignment_check(
        image_path: list, text: str, 
        position: tuple, alignment: str,
        font, font_scale: int, font_color: tuple, font_thickness: int
        ):
    """
    add text as well as where it's aligned. By default it's left aligned from the position
    """
    if alignment == "right":
        (text_w, text_h), _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        position = (position[0]-text_w, position[1]+text_h)
    elif alignment == "center":
        (text_w, text_h), _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        position = (position[0]-text_w//2, position[1]+text_h//2)
    if inline is True:
        return __add_text_to_image_inline()
    else:    
        return __add_text_to_image_path(image_path, text, position, font, font_scale, font_color, font_thickness)


def __add_text_to_image_path(image_path, text, position, font=cv2.FONT_ITALIC, font_scale=1, font_color=(255, 0, 0), font_thickness=50):
    # Read the image
    image = cv2.imread(image_path)

    # Add text to the image
    cv2.putText(image, text, position, font, font_scale, font_color, font_thickness)

    return image

In [29]:
image_path = 'hello_bye.png'
text = 'Hello, World!'

img = cv2.imread(image_path)
h, w, c = img.shape
center_y, center_x = h//2, w//2

# position = (center_y, center_x)  # Position (x, y) where the text will be placed # TODO: write eq for positioning based on image dimension
position = (center_x, center_y) 

## 
result_image = add_text_alignment_check(image_path, text, position, alignment="right", font=cv2.FONT_ITALIC, font_scale=20, font_color=(0, 0, 0), font_thickness=2)
cv2.imwrite('image_with_text_right_aligned.png', result_image)

True

In [30]:
result_image = add_text_alignment_check(image_path, text, position, alignment="left", font=cv2.FONT_ITALIC, font_scale=20, font_color=(0, 0, 0), font_thickness=2)
cv2.imwrite('image_with_text_left_aligned.png', result_image)


True

In [31]:
result_image = add_text_alignment_check(image_path, text, position, alignment="center", font=cv2.FONT_ITALIC, font_scale=20, font_color=(0, 0, 0), font_thickness=2)
cv2.imwrite('image_with_text_center_aligned.png', result_image)

True